In [24]:
import polars as pl
import altair as alt
from prophet import Prophet

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [28]:
!(curl --location 'https://api.core.aql.com/v1/sensors/sensor-data/aggregate/min_ave_max'  \
--header 'Accept: application/json'  \
--header 'Authorization: YOUR_BEARER_TOKEN' \
--header 'Content-Type: application/json'  \
--data '{ "sensor_ids": [ "DEg9FPJkwqj", "yk3Qtn2RKZ6", "KEnkUJMGPO8", "8Y5psrPKW1O", "DE4ESPJkwqj", "4KXMso9vEgY", "ykLJTn2RKZ6", "KE3DhJMGPO8", "pVxnSDOBqmQ", "Q2qkSxlygOB", "8xnNCrPKW1O", "D2DpfPJkwqj", "yDGZCn2RKZ6", "K2xESJMGPO8" ], "startDate":  "2024-03-01 00:00:00", "endDate":  "2024-04-24 00:00:00", "sampleInterval": "Day"}') \
> temp_data.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   356    0    30  100   326    180   1962 --:--:-- --:--:-- --:--:--  2144


In [30]:
!(curl --location 'https://api.core.aql.com/v1/sensors/sensor-data/aggregate/min_ave_max'  \
--header 'Accept: application/json'  \
--header 'Authorization: Bearer YOUR_BEARER_TOKEN' \
--header 'Content-Type: application/json'  \
--data '{ "sensor_ids": [ "QLyjtxlygOB", "wgBEUpVWkK0", "XVEOCJDzj2B", "pVMZSDOBqmQ", "QL4rHxlygOB", "l6rYi7PvmxO", "wgJlIpVWkK0", "XV83tJDzj2B", "DEmDfPJkwqj", "4qDKCo9vEgY", "pqp0IDOBqmQ", "Q2pNSxlygOB", "wA40TpVWkK0", "X0OBHJDzj2B"], "startDate":  "2024-03-01 00:00:00", "endDate":  "2024-04-24 00:00:00", "sampleInterval": "Day"}') \
> humidity_data.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5236k    0 5235k  100   325   269k     16  0:00:20  0:00:19  0:00:01 1239k


In [31]:
temp_data = pl.read_json('temp_data.json')

humidity_data = pl.read_json('humidity_data.json')

SchemaFieldNotFoundError: Average

In [ ]:
temp_data = temp_data.rename({
    "Average":"averageTemp",
    "Maximum":"maximumTemp",
    "Minimum":"minimumTemp"
})

humidity_data = humidity_data.rename({
    "Average":"averageHumidity",
    "Maximum":"maximumHumidity",
    "Minimum":"minimumHumidity"
})

In [ ]:
temp_data.sample(5)

In [ ]:
humidity_data.sample(5)

In [ ]:
temp_chart = alt.Chart(temp_data).mark_line(opacity=0.6).encode(
    x='sensorReadingDate:T',
    y='averageTemp:Q',
    color='sensor_id:N',
    tooltip=['sensorReadingDate:T', 'averageTemp:Q', 'sensor_id:N']
).properties(
    title='Temperature Data',
    width=1300
).interactive()

temp_chart

In [ ]:
humidity_chart = alt.Chart(humidity_data).mark_line(opacity=0.6).encode(
    x='sensorReadingDate:T',
    y='averageHumidity:Q',
    color='sensor_id:N',
    tooltip=['sensorReadingDate:T', 'averageHumidity:Q', 'sensor_id:N']
).properties(
    width=1300
).interactive()
humidity_chart

In [ ]:
t_data = temp_data.select(
 'deviceName',
 'sensorReadingDate',
 'averageTemp'
                ) 

h_data = humidity_data.select(
 'deviceName',
 'sensorReadingDate',
 'averageHumidity')

data = t_data.join(h_data, on=['sensorReadingDate', 'deviceName'])

selection = alt.selection_point(fields=['deviceName'], bind='legend')


alt.Chart(
    data
).mark_circle(
).properties(
    width=800, 
    height=800
).encode(
    alt.X('averageTemp:Q', scale=alt.Scale(zero=False)),
    alt.Y('averageHumidity:Q', scale=alt.Scale(zero=False)),
    color='deviceName:N',
    opacity=alt.condition(selection, alt.value(0.7), alt.value(0.05))
).add_params(
    selection
).interactive()
# what does this tell you about the relationship between temperature and humidity?

# should probably cut out the interpolated readings before doing this. Showing how we do that could be an exercise
# ask them to determine the precision of the thermometer
# maybe create a selection box, then tell the kids to figure out which of the sensors are likely in non-tropical biomes

In [ ]:
 

series = data.filter(
    pl.col.deviceName == data[0,['deviceName']],
).select(
    pl.col.sensorReadingDate.alias('ds'),
    pl.col.averageTemp.alias('y')
).to_pandas()

m = Prophet().fit(series)

future = m.make_future_dataframe(periods=500, freq='h')

forecast = m.predict(future)

predicted_data = alt.Chart(forecast).properties(width=1300).mark_line().encode(x='ds', y='yhat') 

real_data = alt.Chart(series).properties(width=1300).mark_line(color='red').encode(x='ds', y='y') 

predicted_data + real_data
# look at the trend. What is it telling you? what does this suggest about the biomes at Eden?
# change the periods variable to a big number and rerun the code. Can you see any problems with 
# the prediction that your model is now making?